In [ ]:
!pip install tensorflow
!pip install keras
!pip install optuna
!pip install tqdm
!pip install scipy
!pip install pandas
!pip install numpy
!pip install matplotlib
!pip install seaborn
!pip install scikit-learn
!pip install scikit-image
!pip install scikit-learn-intelex
!pip install cartopy

In [8]:
import pandas as pd
import numpy as np
import os
import gc
from scipy.stats import skew, kurtosis, t as student_t, genpareto, energy_distance
from sklearn.neighbors import KernelDensity
from tqdm import tqdm
import tensorflow as tf
from tensorflow.keras import layers, Model, Input
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
import warnings

# Suppress TensorFlow warnings
warnings.filterwarnings('ignore', category=UserWarning, module='tensorflow')

# -------------------- CONFIGURATION --------------------
INPUT_CSV    = os.getenv('INPUT_CSV', 'data/vypocet adapted.csv')
OUTPUT_CSV   = os.getenv('OUTPUT_CSV', 'output/ml_benchmark_risk_table.csv')

# scenario counts
N_SCENARIOS  = 5000

# VAE / Copula parameters
BATCH_SIZE   = 256
EPOCHS       = 100
LATENT_DIM   = 12
STUDENT_DF   = 3       # heavy-tailed latent prior for VAE
EWMA_LAMBDA  = 0.94
KDE_BW       = 0.1
TAIL_QUANT   = 0.99

# PPA constants
PPA_VOLUME   = 1.0
PPA_STRIKE   = 50.0

# -------------------- DATA UTILITIES --------------------
def load_and_clean(path):
    print(f"Loading data from {path}...")
    df0 = pd.read_csv(path, sep=';', header=[0,1], engine='python')
    print("Flattening headers and parsing dates...")
    cols = df0.columns.tolist()
    zones, flat = [], []
    cur = None
    for lvl0, lvl1 in cols:
        if not str(lvl0).startswith('Unnamed'):
            cur = lvl0.replace('-', '_').lower()
        zones.append(None if lvl1.startswith('Unnamed') or lvl1=='Time period' else cur)
    for (lvl0, lvl1), z in zip(cols, zones):
        name = lvl1.replace(' ','_').replace('(','').replace(')','').replace('-','_').lower()
        flat.append(f"{name}_{z}" if z else name)
    df0.columns = flat
    df0['time_period'] = df0['time_period'].str.replace(r'\s*\(.*\)', '', regex=True)
    df0['timestamp'] = pd.to_datetime(
        df0['time_period'].str.split(' - ').str[0], dayfirst=True, errors='coerce')
    df = df0.set_index('timestamp')
    # select columns
    dec_col = 'decoupling_lost_off_taker_vth'
    price_cols = [c for c in flat if 'financial_settlement' in c]
    # convert to numeric
    print("Converting columns to numeric and cleaning NaNs...")
    df[[dec_col] + price_cols] = df[[dec_col] + price_cols].apply(
        lambda s: pd.to_numeric(
            s.astype(str).str.replace('.', '', regex=False).str.replace(',', '.', regex=False),
            errors='coerce'
        )
    )
    df.dropna(subset=[dec_col] + price_cols, inplace=True)
    df = df[(df[price_cols] > 0).all(axis=1)]
    print(f"Data cleaned: {len(df)} rows remain.")
    return df, dec_col, price_cols

# -------------------- METRICS --------------------
def compute_metrics(losses):
    return {
        'Volatility': float(np.std(losses, ddof=0)),
        'Skewness':   float(skew(losses)),
        'Kurtosis':   float(kurtosis(losses, fisher=False))
    }

# -------------------- COPULA SIMULATION --------------------
def simulate_t_copula(returns, n_scen, df, ewma_lambda):
    print("Starting t-Copula simulation...")
    u = returns.rank(axis=0, method='average') / (len(returns) + 1)
    z = student_t.ppf(u, df)
    weights = ewma_lambda ** np.arange(len(z)-1, -1, -1)
    weights /= weights.sum()
    cov = np.cov(z.T, aweights=weights)
    rng = np.random.default_rng(123)
    chi2 = rng.chisquare(df, size=n_scen)
    L = np.linalg.cholesky(cov)
    sims_u = np.empty((n_scen, len(returns), returns.shape[1]))
    for i in tqdm(range(n_scen), desc="Copula u-samples"):
        z0 = rng.standard_normal(z.shape)
        t_val = (z0 @ L.T) / np.sqrt(chi2[i] / df)
        sims_u[i] = student_t.cdf(t_val, df)
    return sims_u

# -------------------- KDE INVERSION --------------------
def invert_kde(u_samples, hist_data, bandwidth):
    vals = np.sort(hist_data)
    kde = KernelDensity(bandwidth=bandwidth).fit(vals[:, None])
    grid = np.linspace(vals.min(), vals.max(), 2000)[:, None]
    logp = kde.score_samples(grid)
    cdf = np.exp(logp).cumsum()
    cdf /= cdf[-1]
    inv = np.interp(u_samples.ravel(), cdf, grid.ravel())
    return inv.reshape(u_samples.shape)

# -------------------- RUN COPULA PIPELINE --------------------
def run_copula(df, price_cols):
    diffs = df[price_cols].diff().dropna()
    sims_u = simulate_t_copula(diffs, N_SCENARIOS, STUDENT_DF, EWMA_LAMBDA)
    sims = np.zeros_like(sims_u)
    print("Inverting copula samples back to returns via KDE...")
    for j in tqdm(range(len(price_cols)), desc="Invert KDE per asset"):
        sims[:, :, j] = invert_kde(sims_u[:, :, j], diffs.iloc[:, j].values, KDE_BW)
    print("Scaling copula marginals to match historical volatility...")
    std_sim = sims.std(axis=(0,1))
    std_hist = diffs.std(axis=0).values
    sims *= (std_hist / std_sim)
    last_prices = df[price_cols].iloc[-1].values
    price_paths = last_prices + np.cumsum(sims, axis=1)
    idx_dec = price_cols.index('financial_settlement_off_taker_pays_vth')
    losses = (PPA_VOLUME * (PPA_STRIKE - price_paths[:, :, idx_dec])).ravel()
    del sims_u, sims; gc.collect()
    return losses

# -------------------- EVT BOOST --------------------
def evt_boost(losses, quantile=TAIL_QUANT):
    print("Applying EVT tail-boost...")
    threshold = np.quantile(losses, quantile)
    excess = losses[losses > threshold] - threshold
    if excess.size > 0:
        shp, loc, scl = genpareto.fit(excess, floc=0)
        boosted = threshold + genpareto.rvs(shp, loc=0, scale=scl, size=excess.size)
        print(f"EVT fit: shape={shp:.2f}, scale={scl:.2f}")
        return np.hstack([losses, boosted])
    print("No extreme exceedances found; skipping EVT.")
    return losses

# -------------------- VAE MODEL --------------------
class StudentTVAE(Model):
    def __init__(self, encoder, decoder, **kwargs):
        super().__init__(**kwargs)
        self.encoder = encoder
        self.decoder = decoder

    def call(self, x):
        m, lv, z = self.encoder(x)
        recon = self.decoder(z)
        recon_loss = tf.reduce_mean(tf.square(x - recon))
        kl_loss = 0.5 * tf.reduce_sum(tf.exp(lv) + m**2 - 1 - lv, axis=1)
        self.add_loss(recon_loss + tf.reduce_mean(kl_loss))
        return recon

def build_vae(n_assets, prior_df):
    print("Building Student-T VAE...")
    inp = Input((n_assets,))
    x = layers.Dense(128, activation='relu')(inp)
    x = layers.BatchNormalization()(x)
    x = layers.Dense(64, activation='relu')(x)
    m = layers.Dense(LATENT_DIM)(x)
    lv = layers.Dense(LATENT_DIM)(x)
    def sample_latent(args):
        mu, logvar = args
        eps = tf.random.normal(tf.shape(mu))
        return mu + tf.exp(0.5 * logvar) * eps
    z = layers.Lambda(sample_latent)([m, lv])
    encoder = Model(inp, [m, lv, z], name='encoder')

    latent_in = Input((LATENT_DIM,))
    y = layers.Dense(64, activation='relu')(latent_in)
    y = layers.BatchNormalization()(y)
    y = layers.Dense(128, activation='relu')(y)
    out = layers.Dense(n_assets)(y)
    decoder = Model(latent_in, out, name='decoder')

    vae = StudentTVAE(encoder, decoder)
    vae.compile(optimizer='adam')
    return vae, encoder, decoder

# -------------------- RUN VAE PIPELINE --------------------
def run_vae(df, price_cols):
    diffs = df[price_cols].diff().dropna()
    print("Training VAE on return differences...")
    vae, enc, dec = build_vae(diffs.shape[1], STUDENT_DF)
    es = EarlyStopping(monitor='val_loss', patience=7, restore_best_weights=True)
    rlr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=4)
    vae.fit(
        diffs.values,
        epochs=EPOCHS,
        batch_size=BATCH_SIZE,
        validation_split=0.1,
        callbacks=[es, rlr],
        verbose=1
    )

    # sample latent from Student-t prior in batches to save memory
    print("Sampling from Student-t prior and decoding in batches...")
    total = N_SCENARIOS * len(diffs)
    rng = np.random.default_rng(42)
    sims_chunks = []
    CHUNK_SIZE = 1_000_000  # adjust based on available RAM

    for start in range(0, total, CHUNK_SIZE):
        end = min(start + CHUNK_SIZE, total)
        batch_len = end - start

        chi2 = rng.chisquare(STUDENT_DF, size=batch_len).astype(np.float32)
        z_batch = (
            rng.standard_normal((batch_len, LATENT_DIM), dtype=np.float32)
            / np.sqrt(chi2 / STUDENT_DF)[:, None]
        )

        flat_batch = dec.predict(z_batch, batch_size=1024)
        sims_chunks.append(flat_batch.astype(np.float32))

    flat_all = np.vstack(sims_chunks)
    sims = flat_all.reshape(N_SCENARIOS, len(diffs), diffs.shape[1])

    # scale marginals
    std_sim = sims.std(axis=(0,1))
    std_hist = diffs.std(axis=0).values
    sims *= (std_hist / std_sim)

    last_prices = df[price_cols].iloc[-1].values
    price_paths = last_prices + np.cumsum(sims, axis=1)
    idx_dec = price_cols.index('financial_settlement_off_taker_pays_vth')
    losses = (PPA_VOLUME * (PPA_STRIKE - price_paths[:, :, idx_dec])).ravel()

    # cleanup
    del sims, flat_all; tf.keras.backend.clear_session(); gc.collect()
    return losses

# -------------------- MAIN PIPELINE --------------------
def main():
    df, dec_col, price_cols = load_and_clean(INPUT_CSV)
    print("Generating historical metrics...")
    hist_losses = df[dec_col].values

    # Copula + EVT
    cop_losses = run_copula(df, price_cols)
    cop_evt = evt_boost(cop_losses)

    # VAE + EVT
    vae_losses = run_vae(df, price_cols)
    vae_evt = evt_boost(vae_losses)

    # scale final losses to historical vol
    print("Final scaling to historical volatility...")
    hist_vol = np.std(hist_losses)
    cop_evt *= (hist_vol / np.std(cop_evt))
    vae_evt *= (hist_vol / np.std(vae_evt))

    # assemble results
    print("Computing diagnostics...")
    results = pd.DataFrame({
        'Historical': compute_metrics(hist_losses),
        't-Copula+EVT': compute_metrics(cop_evt),
        'StudentT-VAE+EVT': compute_metrics(vae_evt)
    }).T
    results.to_csv(OUTPUT_CSV)
    print(results)
    print(f"Results saved to {OUTPUT_CSV}")

if __name__ == '__main__':
    main()


Loading data from data/vypocet adapted.csv...
Flattening headers and parsing dates...
Converting columns to numeric and cleaning NaNs...
Data cleaned: 8388 rows remain.
Generating historical metrics...
Starting t-Copula simulation...


Copula u-samples: 100%|██████████| 5000/5000 [00:43<00:00, 113.96it/s]


Inverting copula samples back to returns via KDE...


Invert KDE per asset: 100%|██████████| 3/3 [00:06<00:00,  2.23s/it]


Scaling copula marginals to match historical volatility...
Applying EVT tail-boost...
EVT fit: shape=-0.24, scale=738.61
Training VAE on return differences...
Building Student-T VAE...
Epoch 1/100
30/30 ━━━━━━━━━━━━━━━━━━━━ 2s 11ms/step - loss: 5.3347 - val_loss: 4.0530 - learning_rate: 0.0010
Epoch 2/100
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 3.4697 - val_loss: 3.4189 - learning_rate: 0.0010
Epoch 3/100
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 3.1964 - val_loss: 3.2742 - learning_rate: 0.0010
Epoch 4/100
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 2.9417 - val_loss: 3.1708 - learning_rate: 0.0010
Epoch 5/100
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 2.7139 - val_loss: 2.9445 - learning_rate: 0.0010
Epoch 6/100
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 2.5782 - val_loss: 2.8351 - learning_rate: 0.0010
Epoch 7/100
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 2.6992 - val_loss: 2.8768 - learning_rate: 0.0010
Epoch 8/100
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step 

OSError: Cannot save file into a non-existent directory: 'output'